In [1]:
import sys
sys.executable

'd:\\Code-Projects\\University\\Fall2023\\Grad\\LipNet-Assistive-AI\\.conda\\python.exe'

In [2]:
from pytube import YouTube
from moviepy.editor import VideoFileClip
from moviepy.video.fx.all import crop
import os
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
from time import sleep
import pysrt
import re
import string
import subprocess

pygame 2.5.2 (SDL 2.28.3, Python 3.9.18)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
# Function to download video
def download_video(url, id):
        try:
            yt = YouTube(url)

            stream = (
                yt.streams.filter(progressive=True, file_extension="mp4").get_highest_resolution()
            )

            if "a.ar" in yt.captions.keys():
                captions = yt.captions.get_by_language_code("a.ar")
                captions.download(
                    srt=True,
                    output_path=r"data/Video Links Dataset/temp_alignments",
                    title=f"{id}")

                stream.download(
                    filename=fr"data/Video Links Dataset/temp_videos/{id}.mp4"
                )

                return fr"data/Video Links Dataset/temp_videos/{id}.mp4",fr"data/Video Links Dataset/temp_alignments/{id} (a.ar).srt", id

            else:
                print("skipping video no arabic subtitles")
                return False, False, False
            
        except Exception as e:
             print(e)
             return None,None, None


# # Function to cut video into generated video segments
# def cut_video_into_segments(video_path, segment_length=2):

#     # cut the segments into similary named parts as the parent

#     clip = VideoFileClip(video_path)
#     duration = int(clip.duration)
#     for start in range(0, duration, segment_length):
#         end = min(start + segment_length, duration)
#         segment = clip.subclip(start, end)
#         segment_filename = f"output_segment_{start}_{end}.mp4"
#         segment.write_videofile(segment_filename, codec="libx264", audio_codec="aac")
        
def english_to_arabic_numbers(text):

    numbers_dict = {
        '0': '٠',
        '1': '١',
        '2': '٢',
        '3': '٣',
        '4': '٤',
        '5': '٥',
        '6': '٦',
        '7': '٧',
        '8': '٨',
        '9': '٩',
    }
    for eng_num, arabic_num in numbers_dict.items():
        text = text.replace(eng_num, arabic_num)
    return text

def remove_special_char(text):
    for i in string.punctuation:
        text = text.replace(i, " ")

    return text

def remove_repeating_spaces(text):
    cleaned_text = re.sub(r' +', ' ', text)
    return cleaned_text

def check_if_not_arabic(text):
    ar = "ابتثجحخدذرزسشصضطظعغفقكلمنهويىء٠١٢٣٤٥٦٧٨٩ "
    for i in text:
        if i not in ar:
            return True
    return False

def filter_srt_sub(text):
    
    if(len(text) < 2):
        return False

    if(text == "[موسيقى]"):
        return False
    
    text = english_to_arabic_numbers(text)
    text = remove_special_char(text)
    text = remove_repeating_spaces(text)
    
    if(check_if_not_arabic(text)):
        return False
    
    if(len(text) < 2):
        return False
    
    return text

def filter_lip_segment():
    pass

def cut_from_srt_file(video_path, srt_path, output_dir_videos,output_dir_str, id):
    
    subs = pysrt.open(srt_path)
    video = VideoFileClip(video_path)

    os.makedirs(output_dir_videos, exist_ok=True)

    for idx, sub in enumerate(subs):
        text = filter_srt_sub(sub.strip())
        if(text):
            continue

        start_time = sub.start.ordinal /1000.0
        end_time = sub.end.ordinal / 1000     
        print("="*100)
        print(idx,text)
        print("="*100)
        video_part = video.subclip(start_time, end_time)
        output_path = os.path.join(output_dir_videos, f"{id}_{idx+1}.mp4")
        
        with open(output_dir_str+f"/{id}_{idx+1}.align","w") as writer:
            writer.write(text.replace(" ","\n"))

        video_part.write_videofile(output_path)

    video.close()



# Function to filter unacceptable video segments

    # create segments
    # for loop on all the segments

    # discard when
    # more than 1 face detected in any frame
    # more than 0.8 of frames have no face detected
    # lips aren't moving for 0.8 of frames
    # orentation of face not in camera range  for 0.8 of frames 

    # cut when
    # start for massive deviation of lips movment, end when end of deviation

In [4]:
def pipline(url, name):
    fails = 10

    video, alignment, id = download_video(url,name)
    while (video == None) and (fails > 0):
        fails -= 1
        sleep(4)
        video, alignment, id = download_video(url,name)

    if not video:
        return False
    print("="*100)
    print(alignment)
    print("="*100)
    subprocess.run(["go","run",".",alignment])

    cut_from_srt_file(video,alignment+".fixed",r"data/Video Links Dataset/videos",r"data/Video Links Dataset/alignments", id)

    return True
    

In [5]:
df = pd.read_csv("data\Video Links Dataset\Video Links Dataset.csv")
df.head()

,link,name,id,creator
0,https://www.youtube.com/watch?v=s0pbzSC81Kc,كتابة الكلام على الفيديو بشكل تلقائي بالذكاء ا...,0,0
1,https://www.youtube.com/watch?v=FYqNG3zwXL4&t=...,فيلم الصفاره,1,0
2,https://www.youtube.com/watch?v=cAsTfosN79A,فيلم كابتن مصر,2,0
3,https://www.youtube.com/watch?v=SJ6_7W2v8ac,فيلم التجربة الدنماركيه,3,0
4,https://www.youtube.com/watch?v=3r3IOzNNKRM,جت كدا اياد الموجي وساره هاني,4,0


In [6]:
df[df.duplicated('link',keep=False)]

,link,name,id,creator


In [7]:
results = []

In [8]:
status = []

# downloaded
# No CC
# Failed

for row in df.values:
     
     result = pipline(row[0], row[2])
     results.append(result)
     break
df['has_arabic'] = status

In [ ]:
# code in captions.py
    # def xml_caption_to_srt(self, xml_captions: str) -> str:
    #     """Convert xml caption tracks to "SubRip Subtitle (srt)".

    #     :param str xml_captions:
    #     XML formatted caption tracks.
    #     """
    #     segments = []
    #     root = ElementTree.fromstring(xml_captions)
    #     i=0
    #     for child in list(root.iter("body"))[0]:
    #         if child.tag == 'p':
    #             caption = ''
    #             if len(list(child))==0:
    #                 # instead of 'continue'
    #                 caption = child.text
    #             for s in list(child):
    #                 if s.tag == 's':
    #                     caption += ' ' + s.text
    #             caption = unescape(caption.replace("\n", " ").replace("  ", " "),)
    #             try:
    #                 duration = float(child.attrib["d"])/1000.0
    #             except KeyError:
    #                 duration = 0.0
    #             start = float(child.attrib["t"])/1000.0
    #             end = start + duration
    #             sequence_number = i + 1  # convert from 0-indexed to 1.
    #             line = "{seq}\n{start} --> {end}\n{text}\n".format(
    #                 seq=sequence_number,
    #                 start=self.float_to_srt_time_format(start),
    #                 end=self.float_to_srt_time_format(end),
    #                 text=caption,
    #             )
    #             segments.append(line)
    #             i += 1
    #     return "\n".join(segments).strip()